In [2]:
import math
import numpy as np
import pandas as pd
import altair as alt
import seaborn as sns
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [70]:
army = pd.read_csv('army_data_dh_masters.csv')
print('Initial', army['relative_position'].min(), army['relative_position'].max())
# army['weighted_position'] = (army['position_proportion'] * 2) - 1
# print('First weighting (x2, -1)', army['weighted_position'].min(), army['weighted_position'].max())

# army['weighted_position'] = np.sign(army['weighted_position']) * (army['weighted_position'] * (army['weighted_position'] + (1 * np.sign(army['weighted_position']))))
# army['weighted_position'] = (np.sign(army['weighted_position']) * ((((np.sign(army['weighted_position']) * army['weighted_position'])) + 1) ** 2)) - army['weighted_position'] - np.sign(army['weighted_position'])
army['weighted_position'] = 1 / (1 + (math.e ** (-8 * (army['relative_position'] - 0.5))))
print('Second weighting', army['weighted_position'].min(), army['weighted_position'].max())

# army['weighted_position'] = (army['weighted_position'] + 2) / 4
# print('Reverted weighting', army['weighted_position'].min(), army['weighted_position'].max())

print(len(army))

# army['weighted_position'] = army['relative_position']
army.head()

Initial 0.008212313037805452 0.9926470588235294
Second weighting 0.019184127653251143 0.9809448058304903
50276


,name,race,matchup,relative_position,army_x,army_y,total_army_value,died_army_value,gameloop,weighted_position
0,TIME,Terran,TvZ,0.955048,131,63,0,0,3360,0.974429
1,正义的泉水沟,Zerg,TvZ,0.034719,138,58,0,0,3360,0.023609
2,TIME,Terran,TvZ,0.903514,133,73,0,0,3600,0.961879
3,正义的泉水沟,Zerg,TvZ,0.023570,135,33,75,0,3600,0.021638
4,TIME,Terran,TvZ,0.952438,132,79,0,0,3840,0.973903


In [73]:
c0 = alt.Chart(army).mark_bar().encode(
    x=alt.X('relative_position:Q', bin=True),
    y='count()',
)
b0 = alt.Chart(army).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)

filtered_army = army[
    (army['total_army_value'] > 2000)
    # & (army['died_army_value'] > (army['total_army_value'] * 0.2))
]

c10 = alt.Chart(filtered_army).mark_bar().encode(
    x=alt.X('relative_position:Q', bin=True),
    y='count()',
)
b10 = alt.Chart(filtered_army).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)

c1 = alt.Chart(army).mark_bar().encode(
    x=alt.X('total_army_value:Q', bin=True),
    y='count()',
)
c2 = alt.Chart(army[army['total_army_value'] < 2000]).mark_bar().encode(
    x=alt.X('total_army_value:Q', bin=True),
    y='count()',
)
d2 = alt.Chart(army[army['total_army_value'] > 2000]).mark_bar().encode(
    x=alt.X('total_army_value:Q', bin=True),
    y='count()',
)

c3 = alt.Chart(army).mark_bar().encode(
    x=alt.X('died_army_value:Q', bin=True),
    y='count()'
)
c5 = alt.Chart(army[army['died_army_value'] < 1000]).mark_bar().encode(
    x=alt.X('died_army_value:Q', bin=True),
    y='count()',
)

c4 = alt.Chart(army).mark_circle(size=60).encode(
    x='army_x',
    y='army_y',
)

protoss = alt.Chart(army[army['race'] == 'Protoss']).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)
terran = alt.Chart(army[army['race'] == 'Terran']).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)
zerg = alt.Chart(army[army['race'] == 'Zerg']).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)

pvt = alt.Chart(army[(army['race'] == 'Protoss') & (army['matchup'] == 'PvT')]).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)
pvz = alt.Chart(army[(army['race'] == 'Protoss') & (army['matchup'] == 'PvZ')]).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)

tvp = alt.Chart(army[(army['race'] == 'Terran') & (army['matchup'] == 'PvT')]).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)
tvz = alt.Chart(army[(army['race'] == 'Terran') & (army['matchup'] == 'TvZ')]).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)

zvp = alt.Chart(army[(army['race'] == 'Zerg') & (army['matchup'] == 'PvZ')]).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)
zvt = alt.Chart(army[(army['race'] == 'Zerg') & (army['matchup'] == 'TvZ')]).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)

pvt_data = filtered_army[(filtered_army['race'] == 'Protoss') & (filtered_army['matchup'] == 'PvT')]
tvz_data = filtered_army[(filtered_army['race'] == 'Terran') & (filtered_army['matchup'] == 'TvZ')]
zvp_data = filtered_army[(filtered_army['race'] == 'Zerg') & (filtered_army['matchup'] == 'PvZ')]
print(pvt_data['weighted_position'].mean())
print(tvz_data['weighted_position'].mean())
print(zvp_data['weighted_position'].mean())

pvt_filtered = alt.Chart(pvt_data).mark_bar().encode(
    x=alt.X('relative_position:Q', bin=True),
    y='count()',
)
w_pvt_filtered = alt.Chart(pvt_data).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)
tvz_filtered = alt.Chart(tvz_data).mark_bar().encode(
    x=alt.X('relative_position:Q', bin=True),
    y='count()',
)
w_tvz_filtered = alt.Chart(tvz_data).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)
zvp_filtered = alt.Chart(zvp_data).mark_bar().encode(
    x=alt.X('relative_position:Q', bin=True),
    y='count()',
)
w_zvp_filtered = alt.Chart(zvp_data).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)

w_pvt_filtered & w_tvz_filtered & w_zvp_filtered # c0 & b0 & c10 & b10 & c11 & b11 # c2 & d2 & protoss & terran & zerg c1 & c0 & b0 & c3 & c4 & c10 & b10

0.4556891501993258
0.4768265204985604
0.30172514370317904


alt.VConcatChart(...)

In [37]:
position = pd.read_csv('army_position.csv')
position.columns = ['name', 'position']
position.head()

,name,position
0,&lt;TheDOH&gt;<sp/>ZAiN,0.888
1,&lt;Reymor&gt;<sp/>GOReynor,0.863
2,Ziggy,0.735
3,uThermal,0.731
4,PengWin,0.720


In [74]:
sorted_army = filtered_army.groupby(by=['name']).mean().sort_values(by='weighted_position', ascending=False)
print(sorted_army['weighted_position'].head(5))
print(sorted_army['weighted_position'].tail(5))

c1 = alt.Chart(sorted_army).mark_bar().encode(
    x=alt.X('weighted_position:Q', bin=True),
    y='count()',
)
c2 = alt.Chart(sorted_army).mark_bar().encode(
    x=alt.X('relative_position:Q', bin=True),
    y='count()',
)
c1 & c2

name
DPGPartinG    0.825499
Future        0.733231
JieShi        0.705573
goblin        0.687372
SpeCial       0.672126
Name: weighted_position, dtype: float64
name
NXZ         0.228957
Lambo       0.223643
GogojOey    0.209471
RiSky       0.199671
Cham        0.151315
Name: weighted_position, dtype: float64


alt.VConcatChart(...)